In [0]:
dbutils.fs.ls("/FileStore/tables/")

Out[43]: [FileInfo(path='dbfs:/FileStore/tables/accounts.zip', name='accounts.zip', size=5297592, modificationTime=1645025994000),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2019/', name='clinicaltrial_2019/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2019.csv', name='clinicaltrial_2019.csv', size=42400056, modificationTime=1648298404000),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2020.csv', name='clinicaltrial_2020.csv', size=46318151, modificationTime=1648298480000),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2021.csv', name='clinicaltrial_2021.csv', size=50359696, modificationTime=1648298499000),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2021_csv_gz.gz', name='clinicaltrial_2021_csv_gz.gz', size=11921810, modificationTime=1651935392000),
 FileInfo(path='dbfs:/FileStore/tables/data_scrubbing_using_rdds.ipynb', name='data_scrubbing_using_rdds.ipynb', size=16994, modificationTime=1645531555000),
 FileInfo

QUESTION 1

In [0]:
clinical_DF = spark.read.options( delimiter = '|', header= True).csv("/FileStore/tables/clinicaltrial_2021.csv")
clinical_DF.show()

+-----------+--------------------+--------------------+--------+----------+--------------------+----------+--------------------+--------------------+
|         Id|             Sponsor|              Status|   Start|Completion|                Type|Submission|          Conditions|       Interventions|
+-----------+--------------------+--------------------+--------+----------+--------------------+----------+--------------------+--------------------+
|NCT02758028|The University of...|          Recruiting|Aug 2005|  Nov 2021|      Interventional|  Apr 2016|                null|                null|
|NCT02751957|     Duke University|           Completed|Jul 2016|  Jul 2020|      Interventional|  Apr 2016|Autistic Disorder...|                null|
|NCT02758483|Universidade Fede...|           Completed|Mar 2017|  Jan 2018|      Interventional|  Apr 2016|   Diabetes Mellitus|                null|
|NCT02759848|Istanbul Medeniye...|           Completed|Jan 2012|  Dec 2014|       Observational|  Ma

In [0]:
clinical_DF.distinct().count()

Out[45]: 387261

QUESTION 2

In [0]:
Types = clinical_DF.filter("Type != ''").groupby("Type").count( ).orderBy("count", ascending = False) 
Types.show()

+--------------------+------+
|                Type| count|
+--------------------+------+
|      Interventional|301472|
|       Observational| 77540|
|Observational [Pa...|  8180|
|     Expanded Access|    69|
+--------------------+------+



Question 3

In [0]:
from pyspark.sql.functions import *

In [0]:
Conditions_1 = clinical_DF.select(explode(split(col("Conditions"), ",")).alias("Conditions")).na.drop()
Conditions_1.show(10, truncate = False)

+------------------------+
|Conditions              |
+------------------------+
|Autistic Disorder       |
|Autism Spectrum Disorder|
|Diabetes Mellitus       |
|Tuberculosis            |
|Lung Diseases           |
|Pulmonary Disease       |
|Diverticular Diseases   |
|Diverticulum            |
|Diverticulosis          |
|Asthma                  |
+------------------------+
only showing top 10 rows



In [0]:
Conditions = Conditions_1.groupBy("Conditions").count().orderBy('count', ascending= False)
Conditions.show(5)

+-----------------+-----+
|       Conditions|count|
+-----------------+-----+
|        Carcinoma|13389|
|Diabetes Mellitus|11080|
|        Neoplasms| 9371|
| Breast Neoplasms| 8640|
|         Syndrome| 8032|
+-----------------+-----+
only showing top 5 rows



Question 4

In [0]:
Mesh_DataFrame = spark.read.options( delimiter = ',', header= True).csv("/FileStore/tables/mesh.csv")
Mesh_DataFrame.show(10, truncate = False)

+----------+-------------------+
|term      |tree               |
+----------+-------------------+
|Calcimycin|D03.633.100.221.173|
|A-23187   |D03.633.100.221.173|
|Temefos   |D02.705.400.625.800|
|Temefos   |D02.705.539.345.800|
|Temefos   |D02.886.300.692.800|
|Abate     |D02.705.400.625.800|
|Abate     |D02.705.539.345.800|
|Abate     |D02.886.300.692.800|
|Difos     |D02.705.400.625.800|
|Difos     |D02.705.539.345.800|
+----------+-------------------+
only showing top 10 rows



In [0]:
tree = Mesh_DataFrame.withColumn("tree", substring("tree", 0,3))
tree.show(10, truncate = False)

+----------+----+
|term      |tree|
+----------+----+
|Calcimycin|D03 |
|A-23187   |D03 |
|Temefos   |D02 |
|Temefos   |D02 |
|Temefos   |D02 |
|Abate     |D02 |
|Abate     |D02 |
|Abate     |D02 |
|Difos     |D02 |
|Difos     |D02 |
+----------+----+
only showing top 10 rows



In [0]:
Combined_DataFrame = tree.join(Conditions_1, tree.term == Conditions_1.Conditions).select(tree["*"], Conditions_1["*"])
Combined_DataFrame.show(20, truncate = False)

+------------------------+----+------------------------+
|term                    |tree|Conditions              |
+------------------------+----+------------------------+
|Autistic Disorder       |F03 |Autistic Disorder       |
|Autism Spectrum Disorder|F03 |Autism Spectrum Disorder|
|Diabetes Mellitus       |C19 |Diabetes Mellitus       |
|Diabetes Mellitus       |C18 |Diabetes Mellitus       |
|Tuberculosis            |C01 |Tuberculosis            |
|Lung Diseases           |C08 |Lung Diseases           |
|Diverticular Diseases   |C06 |Diverticular Diseases   |
|Diverticulum            |C23 |Diverticulum            |
|Diverticulum            |C06 |Diverticulum            |
|Diverticulosis          |C23 |Diverticulosis          |
|Diverticulosis          |C06 |Diverticulosis          |
|Asthma                  |C20 |Asthma                  |
|Asthma                  |C08 |Asthma                  |
|Asthma                  |C08 |Asthma                  |
|Asthma                  |C08 |

In [0]:
Combined_DataFrame2  = Combined_DataFrame.select("tree").na.drop().groupBy("tree").count().orderBy("count", ascending = False)
Combined_DataFrame2.show(5, truncate = False)

+----+------+
|tree|count |
+----+------+
|C04 |143994|
|C23 |136079|
|C01 |106674|
|C14 |94523 |
|C10 |92310 |
+----+------+
only showing top 5 rows



QUESTION 5

In [0]:
dbutils.fs.head("/FileStore/tables/pharma.csv")

[Truncated to first 65536 bytes]
Out[54]: '"Company","Parent_Company","Penalty_Amount","Subtraction_From_Penalty","Penalty_Amount_Adjusted_For_Eliminating_Multiple_Counting","Penalty_Year","Penalty_Date","Offense_Group","Primary_Offense","Secondary_Offense","Description","Level_of_Government","Action_Type","Agency","Civil/Criminal","Prosecution_Agreement","Court","Case_ID","Private_Litigation_Case_Title","Lawsuit_Resolution","Facility_State","City","Address","Zip","NAICS_Code","NAICS_Translation","HQ_Country_of_Parent","HQ_State_of_Parent","Ownership_Structure","Parent_Company_Stock_Ticker","Major_Industry_of_Parent","Specific_Industry_of_Parent","Info_Source","Notes"\n"Abbott Laboratories","Abbott Laboratories","$5,475,000","$0","$5,475,000","2013","20131227","government-contracting-related offenses","False Claims Act and related","kickbacks and bribery","Abbott Laboratories agreed to $5.475 million to resolve allegations that it violated the False Claims Act by paying kickbacks to in

In [0]:
Pharma_DataFrame = spark.read.options( delimiter = ',', header= True).csv("/FileStore/tables/pharma.csv")
Pharma_DataFrame.show(10)

+--------------------+-------------------+--------------+------------------------+---------------------------------------------------------+------------+------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------+--------------------+------------------+---------------------+-----+---------------+-----------------------------+------------------+--------------+--------+--------------------+-----+----------+--------------------+--------------------+------------------+-------------------+---------------------------+------------------------+---------------------------+--------------------+--------------------+
|             Company|     Parent_Company|Penalty_Amount|Subtraction_From_Penalty|Penalty_Amount_Adjusted_For_Eliminating_Multiple_Counting|Penalty_Year|Penalty_Date|       Offense_Group|     Primary_Offense|   Secondary_Offense|         Description|Level_of_Government|  Action_Type|              Agency|    Civil/C

In [0]:
ClinicalPharma= clinical_DF.join(Pharma_DataFrame, clinical_DF.Sponsor == Pharma_DataFrame.Parent_Company, "left")
ClinicalPharma.show(10)

+-----------+--------------------+--------------------+--------+----------+--------------------+----------+--------------------+--------------------+-------+--------------+--------------+------------------------+---------------------------------------------------------+------------+------------+-------------+---------------+-----------------+-----------+-------------------+-----------+------+--------------+---------------------+-----+-------+-----------------------------+------------------+--------------+----+-------+----+----------+-----------------+--------------------+------------------+-------------------+---------------------------+------------------------+---------------------------+-----------+-----+
|         Id|             Sponsor|              Status|   Start|Completion|                Type|Submission|          Conditions|       Interventions|Company|Parent_Company|Penalty_Amount|Subtraction_From_Penalty|Penalty_Amount_Adjusted_For_Eliminating_Multiple_Counting|Penalty_Year|

In [0]:
ClinicalPharma2 = ClinicalPharma.select("Sponsor" , "Parent_Company").where("Parent_Company is null").groupBy(["Sponsor"]).count().orderBy("count", ascending =False)
ClinicalPharma2.show(10)

+--------------------+-----+
|             Sponsor|count|
+--------------------+-----+
|National Cancer I...| 3218|
|M.D. Anderson Can...| 2414|
|Assistance Publiq...| 2369|
|         Mayo Clinic| 2300|
|Merck Sharp & Doh...| 2243|
|   Assiut University| 2154|
|Novartis Pharmace...| 2088|
|Massachusetts Gen...| 1971|
|    Cairo University| 1928|
|   Hoffmann-La Roche| 1828|
+--------------------+-----+
only showing top 10 rows



QUESTION 6

In [0]:
Completed_Studies = clinical_DF.filter("Status == 'Completed'")
Completed_Studies.show(10)

+-----------+--------------------+---------+--------+----------+--------------+----------+--------------------+--------------------+
|         Id|             Sponsor|   Status|   Start|Completion|          Type|Submission|          Conditions|       Interventions|
+-----------+--------------------+---------+--------+----------+--------------+----------+--------------------+--------------------+
|NCT02751957|     Duke University|Completed|Jul 2016|  Jul 2020|Interventional|  Apr 2016|Autistic Disorder...|                null|
|NCT02758483|Universidade Fede...|Completed|Mar 2017|  Jan 2018|Interventional|  Apr 2016|   Diabetes Mellitus|                null|
|NCT02759848|Istanbul Medeniye...|Completed|Jan 2012|  Dec 2014| Observational|  May 2016|Tuberculosis,Lung...|                null|
|NCT02757209|Consorzio Futuro ...|Completed|Apr 2016|  Jan 2018|Interventional|  Apr 2016|              Asthma|Fluticasone,Xhanc...|
|NCT02757508|Washington Univer...|Completed|Mar 2016|  Jul 2017|Inter

In [0]:
CompletedMonth = clinical_DF.filter(clinical_DF.Status == 'Completed').filter(clinical_DF.Completion.endswith('2021')).groupBy('Completion').count().withColumn('Completion', split(col('Completion'), ' ').getItem(0)).sort('Completion', inplace = True)
CompletedMonth.collect()

Out[59]: [Row(Completion='Apr', count=967),
 Row(Completion='Aug', count=700),
 Row(Completion='Feb', count=934),
 Row(Completion='Jan', count=1131),
 Row(Completion='Jul', count=819),
 Row(Completion='Jun', count=1094),
 Row(Completion='Mar', count=1227),
 Row(Completion='May', count=984),
 Row(Completion='Oct', count=187),
 Row(Completion='Sep', count=528)]

In [0]:
from pyspark.sql.functions import unix_timestamp
Months_Completed_Studies = CompletedMonth.sort(unix_timestamp(col('Completion'), 'MMM'))
Months_Completed_Studies2.show()

+----------+-----+
|Completion|count|
+----------+-----+
|       Jan| 1131|
|       Feb|  934|
|       Mar| 1227|
|       Apr|  967|
|       May|  984|
|       Jun| 1094|
|       Jul|  819|
|       Aug|  700|
|       Sep|  528|
|       Oct|  187|
+----------+-----+

